# Goal: To create a time series model that will predict future prepurchase 12 months in advance, at the state level

# Imports

In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import utils

In [2]:
df = pd.read_csv('./data/state_level_metrics.csv', parse_dates = ['index'], index_col = 'index')

In [3]:
states = ['alabama', 'alaska', 'arizona', 'arkansas', 'california', 'colorado', 'connecticut', 'delaware', 'district_of_columbia', 'florida', 'georgia', 'hawaii', 'idaho', 'illinois', 'indiana', 'iowa', 'kansas', 'kentucky', 'louisiana', 'maine', 'maryland', 'massachusetts', 'michigan', 'minnesota', 
              'mississippi', 'missouri', 'montana', 'nebraska', 'nevada', 'new_hampshire', 'new_jersey', 'new_mexico', 'new_york', 'north_carolina', 'north_dakota', 'ohio', 'oklahoma', 'oregon', 'pennsylvania', 'rhode_island', 'south_carolina', 'south_dakota', 'tennessee', 'texas', 'utah', 
              'vermont', 'virginia', 'washington', 'west_virginia', 'wisconsin', 'wyoming']

# Minor cleaning

In [4]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [5]:
#choosing the index interval
def set_intervals(df, interval = str):
    df = df.resample(interval).ffill().interpolate('linear').dropna()
    return df

In [6]:
df = set_intervals(df, 'M')

#### Only using certain features to predict prepurchase. If we used the same features to predict delinquencies and prepurchase, the model would be the same and interpretability becomes even more difficult.

In [11]:
# Removing features that we don't want to use for delinquency prediction
df = df[df.columns.drop(list(df.filter(regex='shelter|population|rate')))]

In [12]:
def state_indices(df):
    """Returns a dictionary of dataframes separating features by state. Each value in the dictionary is a state-represented dataframe that will be iterated over later."""
    df_dict = {}
    df_list = []

    states = ['alabama', 'alaska', 'arizona', 'arkansas', 'california', 'colorado', 'connecticut', 'delaware', 'district_of_columbia', 'florida', 'georgia', 'hawaii', 'idaho', 'illinois', 'indiana', 'iowa', 'kansas', 'kentucky', 'louisiana', 'maine', 'maryland', 'massachusetts', 'michigan', 'minnesota', 
              'mississippi', 'missouri', 'montana', 'nebraska', 'nevada', 'new_hampshire', 'new_jersey', 'new_mexico', 'new_york', 'north_carolina', 'north_dakota', 'ohio', 'oklahoma', 'oregon', 'pennsylvania', 'rhode_island', 'south_carolina', 'south_dakota', 'tennessee', 'texas', 'utah', 
              'vermont', 'virginia', 'washington', 'west_virginia', 'wisconsin', 'wyoming']
    
    for state in states:
        df_list.append(df.filter(regex=f'{state}', axis = 1))
    
    for i in range(len(states)):
        df_dict[states[i]] = df_list[i]

    return df_dict

In [13]:
df_dict = state_indices(df)

In [15]:
# Removing west virginia and arkansas from virginia and kansas dataframes
df_dict['virginia'] = df_dict['virginia'][df_dict['virginia'].columns.drop(list(df_dict['virginia'].filter(regex='west_virginia')))]
df_dict['kansas'] = df_dict['kansas'][df_dict['kansas'].columns.drop(list(df_dict['kansas'].filter(regex='arkansas')))]

In [16]:
for state, df in df_dict.items():
    df_dict[state] = utils.create_indices(df)

In [17]:
# Code written by Joseph Nelson
def interpret_dftest(dftest):
    dfoutput = pd.Series(dftest[0:2], index=['Test Statistic', 'p-value'])
    return dfoutput

# achieve stationarity for all columns, can only do it iteratively for 1 diff without going into recursion or something more complex
# also the var model won't run if we do full stationarity, columns result in having constants
def stationarity(df):
    for col in df:
        df[col] = df[col].diff()
    return df

In [18]:
for state, df in df_dict.items():
    df_dict[state] = stationarity(df).fillna(0)
    # Removes any columns with constant zeros
    df_dict[state] = df_dict[state].loc[:, (df_dict[state] != df_dict[state].iloc[0]).any()]

In [22]:
def var_model(dictionary):    
    forecast_df = pd.DataFrame(columns=[
        'state', 
        'current', 
        'forecast', 
        'rmse_baseline', 
        'rmse_train', 
        'rmse_test', 
        'mae_baseline', 
        'mae_train', 
        'mae_test',
        'mape_baseline',
        'mape_train',
        'mape_test'])

    for state, df in dictionary.items():

        train, test = train_test_split(
        df,
        test_size=0.2,
        shuffle=False)

        model = VAR(train)
        ts_model = model.fit(maxlags=1, ic='aic')  

        # VAR Model Eval Metrics
        train_rmse = np.sqrt(mean_squared_error(train.values, ts_model.forecast(train.values, len(train))))
        test_rmse = np.sqrt(mean_squared_error(test.values, ts_model.forecast(test.values, len(test))))
        train_mae = mean_absolute_error(train.values, ts_model.forecast(train.values, len(train)))
        test_mae = mean_absolute_error(test.values, ts_model.forecast(test.values, len(test)))
        train_mape = mean_absolute_percentage_error(train.values, ts_model.forecast(train.values, len(train)))
        test_mape = mean_absolute_percentage_error(test.values, ts_model.forecast(test.values, len(test)))

        # Baseline (12 month rolling average) Eval Metrics
        # Using a 12 month rolling average as a baseline as we can't use mean the same way as we would if we were doing a standard regression task
        baseline_df = df[-29:].rolling(12).sum().fillna(0)
        baseline_rmse = np.sqrt(mean_squared_error(test.values, baseline_df))
        baseline_mae = mean_absolute_error(test.values, baseline_df)
        baseline_mape = mean_absolute_percentage_error(test.values, baseline_df)
        
        # Add values to df
        forecast_df.loc[state] = [
            state, # state
            df['made_index'][-1], # current
            ts_model.forecast(test.values, 12)[11][3], # forecast
            round(baseline_rmse, 2), # rmse_baseline
            round(train_rmse, 2), # rmse_train
            round(test_rmse, 2), # rmse_test
            round(baseline_mae, 2), # mae_baseline
            round(train_mae, 2), # mae_train
            round(test_mae, 2), # mae_test 
            round(baseline_mape, 2), # mape_baseline
            round(train_mape, 2), # mape_train
            round(test_mape, 2), # mape_test 
        ] # rounding to prevent scientific notation, readability
        # Forecasting 12 months ahead as the dataframe's time intervals are monthly

    return forecast_df.reset_index().drop(columns=['index'])

In [24]:
forecast_df = var_model(df_dict)
# Output to csv for Tableau integration
forecast_df.to_csv('./data/prepurchase_join_to_hud_data.csv')